In [ ]:
#   *   client        server
#  ORM  Python       (R)DBMS ----  Database
#       sqlite3       SQLite         FILE
#              connect
#       cursor(SQL, params)
#       <----------------> Business Logit 1= Dev
#       => SNS(포스팅 등록 및 수정)

In [ ]:
import sqlite3

In [ ]:
con = sqlite3.connect('sns.db')
cur = con.cursor()

In [ ]:
# Table 3개
# Post          Hashtag        PostHashtag(n:m)
# PK, Content   PK, Tag, Cnt   P.PK, H.PK
cur.executescript('''
  DROP TABLE IF EXISTS POST;
  CREATE TABLE POST(
    PNO INTEGER PRIMARY KEY,
    CONTENT TEXT NOT NULL
  );

  DROP TABLE IF EXISTS HASHTAG;
  CREATE TABLE HASHTAG(
    HNO INTEGER PRIMARY KEY,
    NAME TEXT NOT NULL,
    CNT INTEGER NOT NULL DEFAULT 0
  );

  DROP TABLE IF EXISTS PH;
  CREATE TABLE PH(
    PNO INTEGER NOT NULL,
    HNO INTEGER NOT NULL
  );
''')

In [ ]:
# 새로운 게시물 들록 (내용 해시태그들)
# 해시태그: (1)태그풀에 존재하는 태그만, (2)존재하지 않으면 새롭게 추가.
def findTag(tag):
    cur.execute('SELECT HNO FROM HASHTAG WHERE NAME=?', (tag,))
    result = cur.fetchone() # fetchall => fetchone
    if not result: # result가 None 일때
        return 0
    else: # None이 아닐 때, fetchone이니깐 tuple(열, 열, ...)
        return result[0]

def addTag(tag):
    cur.execute('INSERT INTO HASHTAG(NAME) VALUES(?)', (tag,))
    con.commit()
    return cur.lastrowid

In [ ]:
findTag('태그1'), addTag('태그1')

(0, 1)

In [ ]:
findTag('태그1')

1

In [ ]:
def plusCount(hno):
  cur.execute('UPDATE HASHTAG SET CNT = CNT + 1 WHERE HNO=?', (hno,))
  con.commit()

def minusCount(hno):
  cur.execute('UPDATE HASHTAG SET CNT = CNT - 1 WHERE HNO=?', (hno,))
  con.commit()

In [ ]:
plusCount(findTag('태그1')), plusCount(findTag('태그1')), plusCount(findTag('태그1'))

(None, None, None)

In [ ]:
plusCount(findTag('태그2')), plusCount(findTag('태그3'))

(None, None)

In [ ]:
def addPost(content, opt=True, *tags):
    cur.execute('INSERT INTO POST(CONTENT) VALUES(?)', (content,))
    con.commit()
    pno = cur.lastrowid
    # if cur.lastrowid < 1:
    #     return None

    for tag in tags:
        # (1) 태그풀에 존재하는 애들만
        r = findTag(tag)
        # (2) 태그풀에 존재하지 않으면, 추가
        if r == 0 and opt == True:
            r = addTag(tag) # 새로운 태그가 들어가면, return pk
        if r > 0:
            plusCount(r) # 사용된 태그에 +1
            cur.execute('INSERT INTO PH VALUES(?,?)', (pno, r))
            con.commit()

    return pno

In [ ]:
addPost('내용2', False, '태그1', '태그2', '태그3')

1

- Post => 2개
- Hatshtag => 1개
- PH => 2개

In [ ]:
cur.execute('SELECT * FROM POST')
cur.fetchall()

[(1, '내용2')]

In [ ]:
cur.execute('SELECT * FROM HASHTAG')
cur.fetchall()

[(1, '태그1', 4)]

In [ ]:
cur.execute('SELECT * FROM PH')
cur.fetchall()

[(1, 1)]

In [ ]:
addPost('내용3', True, '태그1', '태그2', '태그3')

2

In [ ]:
cur.execute('SELECT * FROM POST')
cur.fetchall()

[(1, '내용2'), (2, '내용3')]

In [ ]:
cur.execute('SELECT * FROM HASHTAG')
cur.fetchall()

[(1, '태그1', 5), (2, '태그2', 1), (3, '태그3', 1)]

In [ ]:
cur.execute('SELECT * FROM PH')
cur.fetchall()

[(1, 1), (2, 1), (2, 2), (2, 3)]

In [ ]:
# 게시글 수정(pno, *tags)
# 태그들 => 기존 태그들과 비교해서, 같으면 냅두고, 다르면 수정(추가/삭제)
def findTagsByID(pno):
    result = list()
    cur.execute('SELECT hno FROM PH WHERE pno=?', (pno,))
    for row in cur.fetchall():
        result.append(row[0])
    return result

def modifyPost(pno, content, *tags):
    cur.execute('UPDATE POST SET CONTENT=? WHERE PNO=?', (content, pno))
    con.commit()

    tlist = findTagsByID(pno) # 원본에 붙어있던 태그들
    hlist = list() # 새롭게 수정된 태그들
    for tag in tags:
        hno = findTag(tag)
        if hno > 0:
            hno = addTag(tag)
        hlist.append(hno)
    removelist = list(set(tlist) - set(hlist))
    pluslist = list(set(hlist) - set(tlist))
    for r in removelist:
        minusCount(r)
        cur.execute('DELETE FROM PH WHERE pno=? AND hno=?', (pno, r))
    for r in pluslist:
        plusCount(r)
        cur.execute('INSERT INTO PH VALUES(?,?)', (pno, r))
    con.commit()

In [ ]:
modifyPost(1, '수정된 내용1', '태그1', '태그2') # 태그1 존치, 태그2 추가
modifyPost(2, '수정된 내용2', '태그3') # 태그1이 삭제, 태그3 추가
modifyPost(3, '수정된 내용3', '태그1', '태그2', '태그3', '태그4') # 태그4가 추가

In [ ]:
# PH        HASHTAG
# 1,1       1,2
# 1,2       2,2
# 2,3       3,2
# 3,1       4,1
# 3,2
# 3,3
# 3,4

In [ ]:
# 실무에서는 절대 이 따구로 하지 말 것
cur.execute('DELETE FROM POST')
cur.execute('DELETE FROM HASHTAG')
cur.execute('DELETE FROM PH')
con.commit()

In [ ]:
# lastrowid => pk, 문제가 생겨 수정한다.
def addTag(tag):
  if findTag(tag) == 0: # 0이면 태그풀이 없는 태그
    cur.execute('INSERT INTO HASHTAG(NAME) VALUES(?)', (tag,))
    con.commit()
  return findTag(tag)

In [ ]:
addPost('내용1', True, '태그1')
addPost('내용2', False, '태그1', '태그2')
addPost('내용3', True, '태그2', '태그3')

3

In [ ]:
cur.execute('SELECT * FROM HASHTAG')
cur.fetchall()

[(1, '태그1', 2), (2, '태그2', 1), (3, '태그3', 1)]

In [ ]:
cur.execute('SELECT * FROM PH')
cur.fetchall()

[(1, 1), (2, 1), (3, 2), (3, 3)]

In [ ]:
modifyPost(1, '수정된내용1', '태그1', '태그3') # 태그3 추가
modifyPost(2, '수정된내용2', '태그2') # 태그1 삭제
modifyPost(3, '수정된내용3') # 태그 삭제

In [ ]:
cur.execute('SELECT * FROM HASHTAG')
cur.fetchall()

[(1, '태그1', 1), (2, '태그2', 1), (3, '태그3', 1)]

In [ ]:
cur.execute('SELECT * FROM PH')
cur.fetchall()

[(1, 1), (1, 3), (2, 2)]

In [ ]:
cur.execute('''
  SELECT A.CONTENT, C.NAME
  FROM POST AS A
  INNER JOIN PH AS B ON B.PNO=A.PNO
  INNER JOIN HASHTAG AS C ON B.HNO=C.HNO
''')
cur.fetchall()

[('수정된내용1', '태그1'), ('수정된내용1', '태그3'), ('수정된내용2', '태그2')]

In [ ]:
def viewPost(pno):
  cur.execute('''
    SELECT NAME FROM HASHTAG
    INNER JOIN PH ON PH.HNO=HASHTAG.HNO AND PH.PNO=?
  ''', (pno,))
  tags = list()
  for row in cur.fetchall():
    tags.append('#'+row[0])
  cur.execute('SELECT CONTENT FROM POST WHERE PNO=?', (pno,))
  content = cur.fetchone()[0]
  return content, ','.join(tags)

viewPost(3)

('수정된내용3', '')

In [ ]:
def search(name, asc=True):
  order = 'ASC' if asc else 'DESC'
  cur.execute('SELECT NAME FROM HASHTAG WHERE NAME LIKE ?', ('%'+name+'%',))
  result = list()
  for row in cur.fetchall():
    result.append(row[0])
  return result

search('태', False)

['태그1', '태그2', '태그3']

In [ ]:
cur.execute('SELECT * FROM HASHTAG')
cur.fetchall()

[(1, '태그1', 1), (2, '태그2', 1), (3, '태그3', 1)]

In [ ]:
def searchPostByTag(tag):
  hno = findTag(search(tag)[0])
  # 중복 없이: DISTINCT
  cur.execute('SELECT DISTINCT(CONTENT) FROM POST INNER JOIN PH ON PH.PNO=POST.PNO AND PH.HNO=?', (hno,))
  print(cur.fetchall())

searchPostByTag('태그1')

[('수정된내용1',)]


백엔드에서 sns를 구현하는 것을 실습을 해봤다.

다음과 같이 기술적인 입장에서 서비스를 제작하려고 하면 내가 무엇을 해야 하는지, 어디를 작업하고 있는지 파악하기 어렵다.

그렇기에 우리는 ORM을 사용하는 것이다.